In [1]:
import mysql.connector
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas import Timestamp
from IPython.display import display
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *
#from Data_with_NLP import *
import spacy
from gensim.models import Word2Vec as w2v
import re
import nltk
import gensim


creds = ["uktuserdaroglu","Vacy4201(Pri","172.20.20.4","hgo",3306]
#Connection to the database
host = creds[2]
user = creds[0]
password = creds[1]
database = creds[3]
port = creds[4]
mydb = mysql.connector.connect(host=host, user=user, database=database, port=port, password=password, auth_plugin='mysql_native_password')
mycursor = mydb.cursor()

#Safecheck to guarantee that the connection worked
mycursor.execute('SHOW TABLES;')
print(f"Tables: {mycursor.fetchall()}")
print(mydb.connection_id) #it'll give connection_id,if got connected

Tables: [('ConsultaUrgencia_doentespedidosconsultaNeurologia2012',), ('consultaneurologia2012',), ('consultaneurologia201216anon_true',), ('hgo_data_032023',)]
899


In [2]:
alertP1 = pd.read_sql("""SELECT * FROM consultaneurologia201216anon_true""",mydb)

/Users/utkuserdaroglu/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
#changing columns with date to a datetime
date_format_alertP1(alertP1)
#change the blank values in proveniencia and priority to unknown
replace_blank(alertP1)
#merge duplicated entity
#entity_duplicated(alertP1)
#changing the text to lower case
#lowering_text(alertP1)
#create a column with the output of a referral and select the refused and accepted ones
alertP1=result(alertP1)

/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['PROVENIENCIA'][alertP1['PROVENIENCIA']=='']='unknown'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['CTH_PRIOR'][alertP1['CTH_PRIOR']=='']='unknown'


In [4]:
alertP1=bef_accepted(alertP1)
class_area(alertP1)
text_length(alertP1)
referral_steps(alertP1)
speciality(alertP1)
alertP1=unit(alertP1)

/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/features.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['step'][alertP1['step']>=3]='3+'


In [5]:
#add symptoms column
#symptoms_column(alertP1)


#add exams columns
#exams(alertP1)
#add comorbidities columns
#comorbidities(alertP1)


#add medication columns
#medication_column(alertP1)



#synonyms(alertP1)
#apply lemmatization and removing stop words
#clean_text(alertP1)
df=alertP1
df.head()

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,mês_saida,ano_saida,Texto,result,before_accepted,area,text_length,step,speciality_type,unit
1499,01-12-1GBF,MIVNNVE,LPHRRVQP,,0,,OUTRA,SAM,,unknown,...,1,2014,Status pos AIT. Estenose carótida interna dir...,1,not accepted before,outside area,51,1,General Neurology,HGO
959,0F-12-1GAG,LLCCECB,LRHHVTQR,GAGBESD,3151571,USF FFMAIS-CS SEIXAL,CS/USF,CTH,,unknown,...,7,2015,Problema(s) de saúde a resolver: Parkinsonismo...,0,not accepted before,inside area,707,1,General Neurology,USF B
868,0F-12-1GAS,LLCJILJ,LQHOOLQV,,0,,OUTRA,SAM,,unknown,...,1,2015,"Ataxia de marcha, desiquilibrio Sind. Meniér...",1,not accepted before,outside area,66,1,General Neurology,HGO
1519,0S-12-1GBF,MIVEMLC,LRHPVSVU,ZHEEZFSC,3151571,USF FFMAIS-CS SEIXAL,CS/USF,CTH,3,Normal,...,11,2015,Envio utente de 89 anos de idade com quadro co...,1,not accepted before,inside area,3024,1,General Neurology,USF B
4844,"A,CBE+SA",VMNNVICN,LQHQLSHL,,0,,OUTRA,SAM,,unknown,...,10,2014,,1,not accepted before,outside area,0,1,General Neurology,HGO


In [6]:
full_text=df.Texto.apply(gensim.utils.simple_preprocess)
full_text

1499    [status, pos, ait, estenose, carótida, interna...
959     [problema, de, saúde, resolver, parkinsonismo,...
868     [ataxia, de, marcha, desiquilibrio, sind, meni...
1519    [envio, utente, de, anos, de, idade, com, quad...
4844                                                   []
                              ...                        
5650                                                   []
6526                                                   []
6528                                                   []
6527                                                   []
1432    [justificação, cefaleias, luísa, rodrigues, dr...
Name: Texto, Length: 9374, dtype: object

In [7]:
model=w2v(full_text,
          min_count=3,
          window=4

)

In [8]:
print(model.wv.most_similar('parkinson'))

[('enxaqueca', 0.9984822869300842), ('sindrome', 0.9978830814361572), ('depressão', 0.9978559017181396), ('vive', 0.9976488351821899), ('eam', 0.9975206255912781), ('enxaquecas', 0.9972448945045471), ('avc', 0.9972030520439148), ('neoplasia', 0.9971941113471985), ('perturbações', 0.9970546364784241), ('tipo', 0.9970267415046692)]


In [9]:
#sentence to vec